In [ ]:
# =============================================================================
# PROGRAM INFERENSI (TESTING) FACE RECOGNITION
# Model: InceptionResnetV1
# Output: Accuracy, Precision, Recall, F1-Score, Confusion Matrix
# =============================================================================

import os
import glob
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import albumentations as A
import pandas as pd

from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from facenet_pytorch import InceptionResnetV1
from tqdm.notebook import tqdm

In [ ]:
# ==========================================
# 2. KONFIGURASI
# ==========================================
CONFIG = {
    'TEST_DATA_DIR': 'Test',
    'MODEL_PATH': 'models\InceptionResnetV1-kfold.pth',
    'IMG_SIZE': 160,       
    'BATCH_SIZE': 32,      
    'DEVICE': 'cuda' if torch.cuda.is_available() else 'cpu'
}

print(f"🔥 Running Inference on: {CONFIG['DEVICE']}")
print(f"📂 Test Data Directory: {CONFIG['TEST_DATA_DIR']}")
print(f"💾 Loading Model from: {CONFIG['MODEL_PATH']}")

In [ ]:
# ==========================================
# 3. DATASET & LOADER CLASS
# ==========================================
class MahasiswaTestDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        path = self.image_paths[idx]
        label = self.labels[idx]

        # Baca Gambar
        image = cv2.imread(path, cv2.IMREAD_COLOR)
        if image is None:
            image = np.zeros((CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE'], 3), dtype=np.uint8)
        else:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Apply Transform
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        return image, torch.tensor(label, dtype=torch.long)

# Transformasi Validasi/Test (Hanya Resize & Normalize)
test_transforms = A.Compose([
    A.Resize(height=CONFIG['IMG_SIZE'], width=CONFIG['IMG_SIZE']),
    # Normalisasi standar FaceNet (Mean 0.5, Std 0.5)
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2()
])

def load_test_data(root_dir):
    image_paths = []
    labels = []
    # Ambil nama kelas dari folder
    classes = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
    class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}

    print(f"🔍 Scanning folder...")
    for cls_name in classes:
        cls_dir = os.path.join(root_dir, cls_name)
        # Support berbagai ekstensi
        files = []
        for ext in ['*.jpg', '*.jpeg', '*.png', '*.webp', '*.JPG', '*.PNG']:
            files.extend(glob.glob(os.path.join(cls_dir, ext)))

        for f in files:
            image_paths.append(f)
            labels.append(class_to_idx[cls_name])

    return np.array(image_paths), np.array(labels), classes

# Load Data
X_test, y_test, class_names = load_test_data(CONFIG['TEST_DATA_DIR'])
print(f"✅ Ditemukan {len(class_names)} kelas.")

# Setup Loader
test_dataset = MahasiswaTestDataset(X_test, y_test, transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=CONFIG['BATCH_SIZE'], shuffle=False, num_workers=2)

In [ ]:
# ==========================================
# 4. LOAD MODEL & WEIGHTS
# ==========================================
# Inisialisasi Arsitektur 
model = InceptionResnetV1(
    pretrained=None, 
    classify=True,
    num_classes=len(class_names),
    dropout_prob=0.6
).to(CONFIG['DEVICE'])

# Load Bobot Model (.pth)
if os.path.exists(CONFIG['MODEL_PATH']):
    state_dict = torch.load(CONFIG['MODEL_PATH'], map_location=CONFIG['DEVICE'])
    model.load_state_dict(state_dict)
    print("✅ Model weights loaded successfully!")
else:
    raise FileNotFoundError(f"❌ File model tidak ditemukan: {CONFIG['MODEL_PATH']}")

model.eval()

In [ ]:
# ==========================================
# 5. PROSES INFERENSI
# ==========================================
all_preds = []
all_labels = []

print("🚀 Memulai proses inferensi...")
with torch.no_grad(): # Matikan gradien untuk menghemat memori
    for images, labels in tqdm(test_loader, desc="Testing"):
        images = images.to(CONFIG['DEVICE'])
        labels = labels.to(CONFIG['DEVICE'])

        # Forward Pass
        outputs = model(images)

        # Ambil prediksi (index kelas dengan probabilitas tertinggi)
        _, preds = torch.max(outputs, 1)

        # Simpan ke list
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

In [ ]:
# ==========================================
# 6. HITUNG METRIK EVALUASI
# ==========================================
print("\n" + "="*50)
print("📊 HASIL EVALUASI MODEL (TEST SET)")
print("="*50)

# 1. Metrik Global (Weighted Average)
acc = accuracy_score(all_labels, all_preds)
prec = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
rec = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

print(f"🏆 Accuracy     : {acc:.2%}")
print(f"🎯 Precision    : {prec:.4f}")
print(f"🔎 Recall       : {rec:.4f}")
print(f"⚖️ F1-Score     : {f1:.4f}")
print("-" * 50)

# 2. Laporan Per Kelas
print("\n📝 Detail Per Kelas:")
print(classification_report(all_labels, all_preds, target_names=class_names, zero_division=0))

# ==========================================
# 7. VISUALISASI CONFUSION MATRIX
# ==========================================
plt.figure(figsize=(20, 20)) # Ukuran besar agar muat 70 kelas
cm = confusion_matrix(all_labels, all_preds)

sns.heatmap(cm, annot=False, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)

plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.title(f'Confusion Matrix\nAccuracy: {acc:.2%}', fontsize=15)
plt.xticks(rotation=90, fontsize=8)
plt.yticks(fontsize=8)
plt.tight_layout()

# Simpan hasil plot (opsional, untuk laporan)
plt.savefig('confusion_matrix_test.png')
plt.show()

In [ ]:
# ==========================================
# 8. PREVIEW PREDIKSI VISUAL
# ==========================================
print("\n🖼️ Preview Sampel Prediksi:")
indices = np.random.choice(len(X_test), min(5, len(X_test)), replace=False)

plt.figure(figsize=(20, 5))
for i, idx in enumerate(indices):
    ax = plt.subplot(1, 5, i + 1)

    # Load gambar original untuk display
    img = cv2.imread(X_test[idx])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    true_cls = class_names[all_labels[idx]]
    pred_cls = class_names[all_preds[idx]]

    color = 'green' if true_cls == pred_cls else 'red'

    plt.imshow(img)
    plt.title(f"True: {true_cls}\nPred: {pred_cls}", color=color, fontsize=10, fontweight='bold')
    plt.axis('off')
plt.show()